In [1]:
using Gridap
using GridapGmsh
using Gridap.Io
using LinearAlgebra

In [2]:
using Pkg
Pkg.add("PrettyTables")
using PrettyTables: pretty_table, ft_printf

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [3]:
function Gridap.DiscreteModelFromFile(filename::AbstractString,::Val{:msh})
  model = GmshDiscreteModel(filename)
  model
end

In [4]:
#the function for the dirichlet boundary conditions as is defined in the excersice sheet
g(x)=sum(x)

g (generic function with 1 method)

In [5]:
 # here it goes the function of the right side of the poisson eq
#this is only a test function, we can chosse whichever and solve the problem
#because here we are trying to measure the run time
f(x) = norm(x)

f (generic function with 1 method)

In [20]:

function solve_poisson(mesh_file; order=1)
  # Load the mesh
  @info "Loading mesh file $mesh_file"
  model = GmshDiscreteModel(mesh_file)
  

  reffe = ReferenceFE(lagrangian, Float64, order)
  V0 = TestFESpace(model, reffe;conformity=:H1, dirichlet_tags=["wall"])
  U = TrialFESpace(V0, g)
    degree = 2 * order 
  Ω = Triangulation(model)
  dΩ = Measure(Ω, degree)

  
  a(u,v) = ∫( ∇(v) ⋅ ∇(u) ) * dΩ
  b(v) = ∫( v*f ) * dΩ
  op = AffineFEOperator(a, b, U, V0)

        A=get_matrix(op)
        bb=get_vector(op)
        A_dense=Matrix(A)
    #mesuring the run time
    time_sparse=  @elapsed begin cholesky(A)\bb end
    time_dense= @elapsed begin lu(A_dense)\bb end
  

#comment this \# if u  dont wanna get the solution file and plot it in paraview  
# for the puorposal of this asigment we can skip the visualization if we want
  #write the output as the entrance to identify each one
     uh = solve(op)
     filename = first(splitext(mesh_file)) * "_solution.vtu"
      writevtk(Ω, filename, cellfields=[ "uh" => uh])
###############################################################
  return [time_sparse, time_dense]
end

solve_poisson (generic function with 1 method)

In [25]:
function ex03(; order =1)
    #here we r making a array with the names of the messhes that we have done in the terminal with gmsh command
    file_name3=["cube1.msh","cube2.msh","cube3.msh","cube4.msh"]
    file_name2=["reg1.msh","reg2.msh","reg3.msh","reg4.msh"]
    file_name1=["line1.msh","line2.msh","line3.msh","line4.msh"]
    
    mesh_sizeCR=[1,0.5,0.3,0.2]
    mesh_sizeL=[0.5,0.4,0.3,0.2]
     
    # we create the variables that will storage our quantities 
    #this ones are for the diference of time
    dtc=zeros(size(file_name3,1))
    dtr=zeros(size(file_name2,1))
    dtl=zeros(size(file_name1,1))
    #this for the time solution for the sparse matrix method
    tsc=zeros(size(file_name3,1))
    tsr=zeros(size(file_name2,1))
    tsl=zeros(size(file_name1,1))
    #this for the time solution for the dense matrix method
    tmc=zeros(size(file_name3,1))
    tmr=zeros(size(file_name2,1))
    tml=zeros(size(file_name1,1))
    
    for i in 1:size(file_name3,1)
        #we call our function solve poisson and we send the mesh file as 
        tsc[i],tmc[i]=solve_poisson(file_name3[i])
        dtc[i]=abs(tsc[i]-tmc[i])
        
        
        #here we will solve for the case of 2dimentions calling the op operator renamed as rec
        #in every iteration we wouldnt save this value because we r only interested on the diference of time
        tsr[i],tmr[i]=solve_poisson(file_name2[i])
        dtr[i]=abs(tsr[i]-tmr[i])
        
        tsl[i],tml[i]=solve_poisson(file_name1[i])
        dtl[i]= abs(tsl[i]-tml[i])
    end
    
    #printing with pretty tables of N=3
      data = hcat(mesh_sizeCR, tsc, tmc,dtc)
      header = ["mesh_size","run-t Sparse sol(N=3)","run-t Dense sol(N=3)","diference time of sol"]
      kwargs = (; header, formatters=(ft_printf("%.2e", [1, 3]), ft_printf("%.2f", [2, 4])))
      pretty_table(data; kwargs...)
      pretty_table(data; kwargs..., backend=Val(:latex))
    #printing with pretty tables of N=2
     data = hcat( mesh_sizeCR,tsr, tmr,dtr)
      header = ["mesh_size", "run-t Sparse sol(N=2)","run-t Dense sol(N=2)","diference time of sol"]
      kwargs = (; header, formatters=(ft_printf("%.2e", [1, 3]), ft_printf("%.2f", [2, 4])))
      pretty_table(data; kwargs...)
      pretty_table(data; kwargs..., backend=Val(:latex))
    #printing with pretty tables of N=1
     data = hcat(mesh_sizeL, tsl, tml,dtl)
      header = ["mesh_size","run-t Sparse sol(N=1)","run-t Dense sol(N=1)","diference time of sol"]
      kwargs = (; header, formatters=(ft_printf("%.2e", [1, 3]), ft_printf("%.2f", [2, 4])))
      pretty_table(data; kwargs...)
      pretty_table(data; kwargs..., backend=Val(:latex))
    return nothing;
    
end

ex03 (generic function with 1 method)

In [26]:
ex03()

Info    : Reading 'cube1.msh'...
Info    : 27 entities
Info    : 336 nodes
Info    : 1173 elements
Info    : Done reading 'cube1.msh'
Info    : Reading 'reg1.msh'...
Info    : 9 entities
Info    : 98 nodes
Info    : 194 elements
Info    : Done reading 'reg1.msh'
Info    : Reading 'line1.msh'...
Info    : 3 entities
Info    : 3 nodes
Info    : 4 elements
Info    : Done reading 'line1.msh'
Info    : Reading 'cube2.msh'...
Info    : 27 entities
Info    : 1843 nodes
Info    : 8089 elements
Info    : Done reading 'cube2.msh'


┌ Info: Loading mesh file cube1.msh
└ @ Main In[20]:4
┌ Info: Loading mesh file reg1.msh
└ @ Main In[20]:4
┌ Info: Loading mesh file line1.msh
└ @ Main In[20]:4
┌ Info: Loading mesh file cube2.msh
└ @ Main In[20]:4


Info    : Reading 'reg2.msh'...
Info    : 9 entities
Info    : 303 nodes
Info    : 604 elements
Info    : Done reading 'reg2.msh'
Info    : Reading 'line2.msh'...
Info    : 3 entities
Info    : 4 nodes
Info    : 5 elements
Info    : Done reading 'line2.msh'


┌ Info: Loading mesh file reg2.msh
└ @ Main In[20]:4
┌ Info: Loading mesh file line2.msh
└ @ Main In[20]:4
┌ Info: Loading mesh file cube3.msh
└ @ Main In[20]:4
┌ Info: Loading mesh file reg3.msh
└ @ Main In[20]:4


Info    : Reading 'cube3.msh'...
Info    : 27 entities
Info    : 7282 nodes
Info    : 36450 elements
Info    : Done reading 'cube3.msh'
Info    : Reading 'reg3.msh'...
Info    : 9 entities
Info    : 729 nodes
Info    : 1456 elements
Info    : Done reading 'reg3.msh'
Info    : Reading 'line3.msh'...
Info    : 3 entities
Info    : 5 nodes
Info    : 6 elements
Info    : Done reading 'line3.msh'
Info    : Reading 'cube4.msh'...
Info    : 27 entities
Info    : 20754 nodes
Info    : 110933 elements
Info    : Done reading 'cube4.msh'


┌ Info: Loading mesh file line3.msh
└ @ Main In[20]:4
┌ Info: Loading mesh file cube4.msh
└ @ Main In[20]:4
┌ Info: Loading mesh file reg4.msh
└ @ Main In[20]:4
┌ Info: Loading mesh file line4.msh
└ @ Main In[20]:4


Info    : Reading 'reg4.msh'...
Info    : 9 entities
Info    : 1601 nodes
Info    : 3200 elements
Info    : Done reading 'reg4.msh'
Info    : Reading 'line4.msh'...
Info    : 3 entities
Info    : 6 nodes
Info    : 7 elements
Info    : Done reading 'line4.msh'
┌───────────┬───────────────────────┬──────────────────────┬───────────────────────┐
│ mesh_size │ run-t Sparse sol(N=3) │ run-t Dense sol(N=3) │ diference time of sol │
├───────────┼───────────────────────┼──────────────────────┼───────────────────────┤
│  1.00e+00 │                  0.00 │             9.37e-04 │                  0.00 │
│  5.00e-01 │                  0.01 │             7.49e-02 │                  0.07 │
│  3.00e-01 │                  0.05 │             2.96e+00 │                  2.91 │
│  2.00e-01 │                  0.33 │             7.55e+01 │                 75.18 │
└───────────┴───────────────────────┴──────────────────────┴───────────────────────┘
\begin{table}
  \begin{tabular}{rrrr}
    \hline\hline
    \